In [ ]:
import numpy as np
from sklearn.datasets import load_digits
import time
import math

#function:calculating pairwise distances
#input:image data NxD array 
#output:distance matrix NxN array
def cal_pairwise_dist(x):
    print("x's shape:",x.shape)
    sum_x = np.sum(np.square(x), 1)
    dist = np.add(np.add(-2 * np.dot(x, x.T), sum_x).T, sum_x)
    x10 = np.percentile(dist, 10)
    #x80 = np.percentile(dist, 80)
    shape = dist.shape
    maxa=dist.max()
    mina=dist.min()
    result = dist
    for x in range(0, shape[0]):
        for y in range(0, shape[1]):
            if dist[x, y] <= x10:
                a=dist[x,y]
                weight=(a-mina+0.1)/(maxa-mina)
                result[x, y] = a*weight          
    dist=result
    return dist


#function:calculating perplexity
#input:distance matrix NxN array 
#output:perplexity,prob
def cal_perplexity(dist, idx=0, beta=1.0):
    prob = np.exp(-dist * beta)
    prob[idx] = 0
    sum_prob = np.sum(prob)
    if sum_prob < 1e-12:
        prob = np.maximum(prob, 1e-12)
        perp = -12
    else:
        perp = np.log(sum_prob) + beta * np.sum(dist * prob) / sum_prob
        prob /= sum_prob
    return perp, prob

#function:search for beta and computation of pairwise prob
#input:image data NxD array
#output:pair_prob
def seach_prob(x, tol=1e-5, perplexity=30.0):
    print("Computing pairwise distances...")
    (n, d) = x.shape
    dist = cal_pairwise_dist(x)
    dist[dist < 0] = 0
    pair_prob = np.zeros((n, n))
    beta = np.ones((n, 1))
    base_perp = np.log(perplexity)

    for i in range(n):
        if i % 500 == 0:
            print("Computing pair_prob for point %s of %s ..." % (i, n))

        betamin = -np.inf
        betamax = np.inf
        perp, this_prob = cal_perplexity(dist[i], i, beta[i])

        perp_diff = perp - base_perp
        tries = 0
        while np.abs(perp_diff) > tol and tries < 50:
            if perp_diff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2
                else:
                    beta[i] = (beta[i] + betamin) / 2

            perp, this_prob = cal_perplexity(dist[i], i, beta[i])
            perp_diff = perp - base_perp
            tries = tries + 1
        pair_prob[i,] = this_prob
    print("Mean value of sigma: ", np.mean(np.sqrt(1 / beta)))

    return pair_prob

#function :dimensionality reduction
#input:image data NxD array,target dimension figure
#output:image data after dimensionality reduction
def WDM_tSNE(x, no_dims=2, perplexity=30.0, max_iter=1000):
    t1=time.time()
    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array x should have type float.")
        return -1

    (n, d) = x.shape

    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 500
    min_gain = 0.01
    y = np.random.randn(n, no_dims)
    dy = np.zeros((n, no_dims))
    iy = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))


    P = seach_prob(x, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)  # pij

    print("T-SNE DURING:%s" % time.clock())
    P = P * 4
    P = np.maximum(P, 1e-12)

    for iter in range(max_iter):
        sum_y = np.sum(np.square(y), 1)
        num = 1 / (1 + np.add(np.add(-2 * np.dot(y, y.T), sum_y).T, sum_y))
        num[range(n), range(n)] = 0
        Q = num / np.sum(num)  # qij
        Q = np.maximum(Q, 1e-12) 

        PQ = P - Q
        for i in range(n):
            dy[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (y[i, :] - y), 0)

        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum

        gains = (gains + 0.2) * ((dy > 0) != (iy > 0)) + (gains * 0.8) * ((dy > 0) == (iy > 0))
        gains[gains < min_gain] = min_gain
        iy = momentum * iy - eta * (gains * dy)
        y = y + iy
        y = y - np.tile(np.mean(y, 0), (n, 1))

        if (iter + 1) % 100 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration ", (iter + 1), ": error is ", C)
        
            if (iter + 1) != 100:
                ratio = C / oldC
                print("ratio ", ratio)
                if ratio >= 0.95:
                    break
            oldC = C
        
        if iter == 100:
            P = P / 4
            
    t2 = time.time()
    print('time:%s' % (t2 - t1))
    print("finished training!")
    return y


if __name__ == "__main__":
       
    #test tomato
    data=np.loadtxt(fname="Tomato_data_3.csv",delimiter=",")
    
    X_tsne = WDM_tSNE(data, 2)
    
